# Data Cleaning

## Introduction

* This notebook goes through a necasssary step of cleaning the data before it is used for exploratory data analysis. 
* The input of this notebook is a training dataset in csv format sourced from Kaggle. https://www.kaggle.com/c/nlp-getting-started
* The output of this notebook is a csv file with clean and lemmatized text data. 

## Reading and Understanding the dataset

In [69]:
# Importing libraries
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re           # regular expression 
import string       # String Handling
import random       #For selecting random rows
import nltk         # Natural langauage processing toolkit
from nltk.stem import WordNetLemmatizer  #Used for Lemmatizing the text
from nltk.corpus import wordnet          #Used for POS tagging 
from nltk.corpus import stopwords        #Stopwords to be removed from text


Reading the training data into a dataframe using pandas and viewing the top 20 rows of data

In [70]:
train_df = pd.read_csv("train.csv")
train_df.head(20)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


First 20 rows in 'Keyword' and 'Location' column are NaNs. Now Checking if there are nulls in other columns of the dataframe.

In [71]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
id          7613 non-null int64
keyword     7552 non-null object
location    5080 non-null object
text        7613 non-null object
target      7613 non-null int64
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


Checking number of unique values in 'keywords' and 'location'.

In [72]:
len(train_df.keyword.unique())

222

In [73]:
len(train_df.location.unique())

3342

A glimpse at 30 rows from the 'locations' column selected randomly


In [74]:
random.sample(list((train_df.location.unique())),k=30)


['im definitely taller than you.',
 'Made Here In Detroit ',
 'Haddonfield, NJ',
 'Pig Symbol, Alabama',
 'at my home',
 'North West London',
 'EGYPT',
 'Georgia, U.S.A.',
 '??????',
 'Ontario, Canada. ',
 'Minneapolis - St. Paul',
 'THE WORLD T.G.G / M.M.M ',
 'Utica NY',
 'Aztec NM',
 'Reading a romance novel',
 'IRAQ',
 'MontrÌ©al, QuÌ©bec',
 '36 & 38',
 'Los Angeles',
 'iamdigitalent.com',
 '?? ??',
 'Cymru araul',
 'playing soccer & eating pizza',
 'planeta H2o',
 'Albuquerque',
 'Warm Heart Of Africa',
 'Sydney, New South Wales',
 'we?it \x89Û¢ ixwin',
 'U.S.A',
 'Wakanda']

A glimpse at 30 rows from the 'keywords' column selected randomly

In [75]:
random.sample(list((train_df.keyword.unique())),k=30)


['natural%20disaster',
 'sunk',
 'volcano',
 'death',
 'destruction',
 'blaze',
 'burning',
 'bombing',
 'earthquake',
 'twister',
 'terrorism',
 'crashed',
 'annihilation',
 'rainstorm',
 'smoke',
 'collided',
 'outbreak',
 'screams',
 'body%20bags',
 'storm',
 'desolate',
 'thunderstorm',
 'drown',
 'detonate',
 'panic',
 'hazard',
 'massacre',
 'disaster',
 'demolish',
 'floods']

A glimpse at 30 rows from the 'text' column selected randomly

In [76]:
# Selecting 30 random from text column randomly
random.sample(list((train_df.text)),k=30)

['The Latest: Washington #Wildfire misses town; evacuations end - KHQ Right Now http://t.co/aNlhW2IzkZ',
 'Reality Training: Train falls off elevated tracks during windstorm http://t.co/JIOMnrCygT #Paramedic #EMS',
 'Watch This Airport Get Swallowed Up By A Sandstorm In Under A Minute http://t.co/VZPKn23RX4',
 "@pmharper don't worry I'm sure climate has nothing to do with hail floods tornados in Alberta &amp; Ontario. I'm sure god's just mad at you.",
 "Madhya Pradesh Train Derailment: Village Youth Saved Many Lives: A group of villagers saved over 70 passengers' lives after two train...",
 'Imagini noi si 2 clipuri The Hobbit: The Desolation of Smaug -... http://t.co/j6CfwUKofE #cliptv #desolationofsmaug #poze #thehobbit',
 'Beautiful lightning as seen from plane window http://t.co/5CwUyLnFUm http://t.co/1tyYqFz13D',
 'Watch This Airport Get Swallowed Up By A Sandstorm In Under A Minute http://t.co/sEquWmvFx4',
 'Battle of the GOATS  https://t.co/ofECs6tcvC',
 'Global precipitation me

## Observation

Some data incosistencies or redundant information found in the dataset are as follows

* Upper case and lower case at unexpected location
* Punctuations
* Numbers in text 
* Use of cities, states and Country names. (Granularity problem)
* Special characters such as \x89ÛÒ and \n
* Hyperklinks
* Tags in tweets




## Cleaning the data

The method below does the following to clean anomalies in 'location' column:

* Changes all the text to lower case
* Removes punctutations
* Removes texts with numbers
* Removes cities names if country/state names are mentioned. (High level granularity is maintained)

In [77]:
# Method to chaange text to lower case and remove punctionation
def cleaning_location(text):
    text = str(text)
    text = text.lower()    #lower case
    text = text.split(',')[-1:][0].strip() # Removing city names when country/state name is present
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  #removing punctuations
    text = re.sub('\w*\d\w*', '', text) #removing text with number
    return text

# Applying the method to location column
train_df.location = train_df.location.apply(cleaning_location)

The method below does the following to clean anomalies in 'keyword' column:

* Changes all the text to lower case
* Removes punctutations
* replaces number with space (as %20 was found in middle of two words)

In [78]:
def cleaning_keyword(text):
    text = str(text)
    text = text.lower()    ##lower case
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  ##removing punctuations
    text = re.sub('(\d+)', ' ', text)   #replacing numbers with space
    return text

# Applying the method to keyword column
train_df.keyword = train_df.keyword.apply(cleaning_keyword)

The following method to clean anomalies in 'text' column does the following

* Changes all the text to lower case
* Removes words starting with @ to remove the tags and mentions example: @barackobama
* Adds a column with hashtag values
* Removes links
* Removes punctuation
* Removes words with numbers
* Removes special characters examples: \x89û,\x89ûó etc 
* Removes '\n' 


In [79]:
def cleaning_text(text):
    
    text = str(text)
    text = text.lower()    ##lower case
    
    text = re.sub(r'@[A-Za-z]+[A-Za-z0-9-_]+', '',text) #removing any word starting with @   \w
    text = re.sub(r'https|www|http\S+', '', text)  #removing any word starting with http
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  ##removing punctuations
    text = re.sub('\w*\d\w*', '', text)  #removing words with numbers
    text = re.sub(r'[^\x00-\x7F]+', '', text) # removing special characters
    text.replace("\n","")
    return text

The following method converts NLTK tags to wordnet tags which would be used to lemmatize the words in the following method

In [80]:
def wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

The following method is used to lemmatize the sentences in the following order:

* Clean the sentences by calling the cleaning_text method on each sentence.
* Tokenizing the sentence and generating a nltk POS tag for each word in the cleaned sentence.
* Converting the nltk POS tag to Wordnet POS tag by calling wordnet_tag method.
* Removes the stopwords
* Lemmatizes the tokens using the pos tags


In [81]:
lemmatizing = WordNetLemmatizer()

def lemmatize_sentence(sentence):
    sentence = cleaning_text(sentence) #Cleaning the sentence
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  #Tokenizing and tagging each word 
    wordnet_tagged = map(lambda x: (x[0], wordnet_tag(x[1])), nltk_tagged)  # Coverting the NLTK tags to wordnet tag
    
    #Lemmatizing the tagged tokens
    lemmatized_sentence = [] #empty list for lemmatized words
    for word, tag in wordnet_tagged:
        if word not in set(stopwords.words('english')):  #removing stopwords
            if tag is None:                   
                lemmatized_sentence.append(word) #adding the word as it is if POS tag missing
            else:        
                #else use the tag to lemmatize the token
                lemmatized_sentence.append(lemmatizing.lemmatize(word, tag))  ##lemmatizing the token using the POS tag
    return " ".join(lemmatized_sentence)



In [82]:
# Applying the lemmatizing method to text column 
train_df.text= train_df.text.apply(lemmatize_sentence)

Let's have a look at the cleaned dataframe

In [83]:
train_df.sample(n=20)

,id,keyword,location,text,target
3456,4944,exploded,jamaica,go replace sarcasm meter explode,0
3450,4934,exploded,usa,tell ten year ago id see anime big screen prob...,0
3640,5189,fatalities,france,estimate damage fatality hiroshimasized atomic...,1
1577,2278,cliff fall,bc,dont let style fall flat summer lord amp cliff...,0
6236,8905,snowstorm,uk,new photo oak snowstorm take winter southdowns...,1
7110,10187,violent storm,costa rica,rt jupiter red spot violent storm large entire...,0
984,1425,body bag,nan,im im say theyre lucky go home family put body...,0
5830,8329,rubble,west africa,tnn china stock market crash gem rubble,1
5888,8409,sandstorm,united states,watch airport get swallow sandstorm minute,1
1284,1852,burned,dc,burn calorie minute walk mph brisk pace myfitn...,0


Checking if text from any row was completely removed due to cleaning

In [84]:
train_df.loc[train_df.text == '']

,id,keyword,location,text,target
5115,7295,nuclear reactor,nan,,0


There is one row with id 7295 where the whole text was removed. 

The original train data is read again to see the content of the row with id 7295

In [85]:
df_crosscheck = pd.read_csv('train.csv')
df_crosscheck.loc[df_crosscheck.id == 7295]

,id,keyword,location,text,target
5115,7295,nuclear%20reactor,NaN,Err:509,0


It can be observed above that the text in that particular row is an error message. Hence, that text was correctly cleaned from that row. 

The row is removed from the dataframe and the dataframe is saved as a csv file to be used for EDA.

In [86]:
train_df = train_df[train_df.text != '']
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7612 entries, 0 to 7612
Data columns (total 5 columns):
id          7612 non-null int64
keyword     7612 non-null object
location    7612 non-null object
text        7612 non-null object
target      7612 non-null int64
dtypes: int64(2), object(3)
memory usage: 356.8+ KB


In [87]:
train_df.to_csv("Clean_train_data.csv")

Cleaning test data using the predefined methods.

In [88]:
test_df = pd.read_csv('test.csv')

In [89]:
# Applying the method to location column
test_df.location = test_df.location.apply(cleaning_location)
# Applying the method to keyword column
test_df.keyword = test_df.keyword.apply(cleaning_keyword)

# Applying the lemmatizing method to text column 
test_df.text= test_df.text.apply(lemmatize_sentence)

In [90]:
test_df.to_csv('Clean_test_data.csv')